This notebooks is created using Chapter 2 of the the [Advanced NLP with spaCy](https://course.spacy.io/en/chapter2) course

# Chapter 2: Large-scale data analysis with spaCy

## Data Structures

### Shared vocab and string store (1)

spaCy stores all shared data in a vocabulary, the Vocab.

This includes words, but also the labels schemes for tags and entities.

To save memory, all strings are encoded to hash IDs. If a word occurs more than once, we don't need to save it every time.

Instead, spaCy uses a hash function to generate an ID and stores the string only once in the string store. The string store is available as `nlp.vocab.strings`.

It's a lookup table that works in both directions. You can look up a string and get its hash, and look up a hash to get its string value. Internally, spaCy only communicates in hash IDs.

Hash IDs can't be reversed, though. If a word is not in the vocabulary, there's no way to get its string. That's why we always need to pass around the shared vocab.

- `Vocab`: stores data shared across multiple documents
- To save memory, spaCy encodes all strings to hash values
- Strings are only stored once in the `StringStore` via `nlp.vocab.strings`
- String store: **lookup** table in both directions

In [2]:
# Import spaCy
import spacy

# Create a blank English nlp object
nlp = spacy.blank("en")

In [3]:
nlp.vocab.strings.add("coffee")
coffee_hash = nlp.vocab.strings["coffee"]
coffee_string = nlp.vocab.strings[coffee_hash]

- Hashes can't be reversed – that's why we need to provide the shared vocab

In [8]:
coffee_hash

3197928453018144401

In [4]:
# Raises an error if we haven't seen the string before
string = nlp.vocab.strings[3197928453018144401]

In [6]:
string

'coffee'

In [7]:
nlp.vocab.strings

### Shared vocab and string store (2)

To get the hash for a string, we can look it up in `nlp.vocab.strings`.

To get the string representation of a hash, we can look up the hash.

A `Doc` object also exposes its vocab and strings.

- Look up the string and hash in `nlp.vocab.strings`

In [9]:
doc = nlp("I love coffee")
print("hash value:", nlp.vocab.strings["coffee"])
print("string value:", nlp.vocab.strings[3197928453018144401])

hash value: 3197928453018144401
string value: coffee


- The `doc` also exposes the vocab and strings

In [10]:
doc = nlp("I love coffee")
print("hash value:", doc.vocab.strings["coffee"])

hash value: 3197928453018144401


In [11]:
print("hash value:", doc.vocab.strings["love"])

hash value: 3702023516439754181


### Lexemes: entries in the vocabulary

Lexemes are context-independent entries in the vocabulary.

You can get a lexeme by looking up a string or a hash ID in the vocab.

Lexemes expose attributes, just like tokens.

They hold context-independent information about a word, like the text, or whether the word consists of alphabetic characters.

Lexemes don't have part-of-speech tags, dependencies or entity labels. Those depend on the context.

- A Lexeme object is an entry in the vocabulary

In [12]:
doc = nlp("I love coffee")
lexeme = nlp.vocab["coffee"]

# Print the lexical attributes
print(lexeme.text, lexeme.orth, lexeme.is_alpha)

coffee 3197928453018144401 True


- Contains the context-independent information about a word
  - Word text: lexeme.text and lexeme.orth (the hash)
  - Lexical attributes like lexeme.is_alpha
  - Not context-dependent part-of-speech tags, dependencies or entity labels

### Vocab, hashes and lexemes

Here's an example.

The Doc contains words in context – in this case, the tokens "I", "love" and "coffee" with their part-of-speech tags and dependencies.

Each token refers to a lexeme, which knows the word's hash ID. To get the string representation of the word, spaCy looks up the hash in the string store.

![Vocab, hashes and lexemes](./img/vocab_stringstore.png)


## Strings to hashes

Part 1
- Look up the string “cat” in `nlp.vocab.strings` to get the hash.
- Look up the hash to get back the string.

In [14]:
import spacy

nlp = spacy.blank("en")
doc = nlp("I have a cat")

# Look up the hash for the word "cat"
cat_hash = nlp.vocab.strings["cat"]
print(cat_hash)

# Look up the cat_hash to get the string
cat_string = nlp.vocab.strings[cat_hash]
print(cat_string)

5439657043933447811
cat


Part 2
- Look up the string label “PERSON” in `nlp.vocab.strings` to get the hash.
- Look up the hash to get back the string.

In [15]:
import spacy

nlp = spacy.blank("en")
doc = nlp("David Bowie is a PERSON")

# Look up the hash for the string label "PERSON"
person_hash = nlp.vocab.strings["PERSON"]
print(person_hash)

# Look up the person_hash to get the string
person_string = nlp.vocab.strings[person_hash]
print(person_string)

380
PERSON


## Vocab, hashes and lexemes

Why does this code throw an error?

In [19]:
import spacy

# Create an English and German nlp object
nlp = spacy.blank("en")
nlp_de = spacy.blank("de")

# Get the ID for the string 'Bowie'
bowie_id = nlp.vocab.strings["Bowie"]
print(bowie_id)

# Look up the ID for "Bowie" in the vocab
print(nlp_de.vocab.strings[bowie_id])

2644858412616767388


KeyError: "[E018] Can't retrieve string for hash '2644858412616767388'. This usually refers to an issue with the `Vocab` or `StringStore`."

The string "Bowie" isn’t in the German vocab, so the hash can’t be resolved in the string store.

Hashes can’t be reversed. To prevent this problem, add the word to the new vocab by processing a text or looking up the string, or use the same vocab to resolve the hash back to a string.

## Data Structures(2)

Now that you know all about the vocabulary and string store, we can take a look at the most important data structure: the `Doc`, and its views `Token` and `Span`.

### The Doc object

The `Doc` is one of the central data structures in spaCy. It's created automatically when you process a text with the `nlp` object. But you can also instantiate the class manually.

After creating the `nlp` object, we can import the `Doc` class from `spacy.tokens`.

Here we're creating a doc from three words. The spaces are a list of boolean values indicating whether the word is followed by a space. Every token includes that information – even the last one!

The `Doc` class takes three arguments: the shared vocab, the words and the spaces.

In [20]:
# Create an nlp object
import spacy
nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)

### The Span object(1)

A `Span` is a slice of a doc consisting of one or more tokens. The `Span` takes at least three arguments: the doc it refers to, and the start and end index of the span. Remember that the end index is exclusive!

![The Span Object](./img/span_indices.png)

### The Span object(2)

To create a `Span` manually, we can also import the class from `spacy.tokens`. We can then instantiate it with the doc and the span's start and end index, and an optional label argument.

The `doc.ents` are writable, so we can add entities manually by overwriting it with a list of spans.

In [23]:
# Import the Doc and Span classes
from spacy.tokens import Doc, Span

# The words and spaces to create the doc from
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Create a doc manually
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Create a span manually
span = Span(doc, 0, 2)

# Create a span with a label
span_with_label = Span(doc, 0, 2, label="GREETING")

# Add span to the doc.ents
doc.ents = [span_with_label]

### Best practices

A few tips and tricks before we get started:

The `Doc` and `Span` are very powerful and optimized for performance. They give you access to all references and relationships of the words and sentences.

If your application needs to output strings, make sure to convert the doc as late as possible. If you do it too early, you'll lose all relationships between the tokens.

To keep things consistent, try to use built-in token attributes wherever possible. For example, `token.i` for the token index.

Also, don't forget to always pass in the shared vocab!

- `Doc` and `Span` are very powerful and hold references and relationships of words and sentences
  - **Convert result to strings as late as possible**
  - **Use token attributes if available** – for example, `token.i` for the token index
  
- Don't forget to pass in the shared `vocab`

## Creating a Doc

Let’s create some Doc objects from scratch!

Part 1

Import the `Doc` from `spacy.tokens`.

Create a `Doc` from the `words` and `spaces`. Don’t forget to pass in the vocab!

In [1]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "spaCy is cool!"
words = ["spaCy", "is", "cool", "!"]
spaces = [True, True, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

spaCy is cool!


Part 2

Import the `Doc` from `spacy.tokens`.

Create a `Doc` from the `words` and `spaces`. Don’t forget to pass in the vocab!

In [2]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "Go, get started!"
words = ["Go", ",", "get", "started", "!"]
spaces = [False, True, True, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Go, get started!


Part 3

Import the `Doc` from `spacy.tokens`.

Complete the `words` and `spaces` to match the desired text and create a `doc`.

In [3]:
import spacy

nlp = spacy.blank("en")

# Import the Doc class
from spacy.tokens import Doc

# Desired text: "Oh, really?!"
words = ["Oh", ",", "really", "?", "!"]
spaces = [False, True, False, False, False]

# Create a Doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

Oh, really?!


## Docs, spans and entities from scratch

In this exercise, you’ll create the `Doc` and `Span` objects manually, and update the named entities – just like spaCy does behind the scenes. A shared `nlp` object has already been created.

- Import the `Doc` and `Span` classes from `spacy.tokens`.
- Use the `Doc` class directly to create a `doc` from the words and spaces.
- Create a `Span` for “David Bowie” from the `doc` and assign it the label `"PERSON"`.
- Overwrite the `doc.ents` with a list of one entity, the “David Bowie” `span`.

In [5]:
import spacy

nlp = spacy.blank("en")

# Import the Doc and Span classes
from spacy.tokens import Doc, Span

words = ["I", "like", "David", "Bowie"]
spaces = [True, True, True, False]

# Create a doc from the words and spaces
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

# Create a span for "David Bowie" from the doc and assign it the label "PERSON"
span = Span(doc, 2, 4, label="PERSON")
print(span.text, span.label_)

# Add the span to the doc's entities
doc.ents = [span]

# Print entities' text and labels
print([(ent.text, ent.label_) for ent in doc.ents])

I like David Bowie
David Bowie PERSON
[('David Bowie', 'PERSON')]


## Data structures best practices

The code in this example is trying to analyze a text and collect all proper nouns that are followed by a verb.

In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Berlin looks like a nice city")

# Get all tokens and part-of-speech tags
token_texts = [token.text for token in doc]
pos_tags = [token.pos_ for token in doc]

for index, pos in enumerate(pos_tags):
    # Check if the current token is a proper noun
    if pos == "PROPN":
        # Check if the next token is a verb
        if pos_tags[index + 1] == "VERB":
            result = token_texts[index]
            print("Found proper noun before a verb:", result)

Found proper noun before a verb: Berlin


Why is the code bad?

It only uses lists of strings instead of native token attributes. This is often less efficient, and can't express complex relationships.

That's correct! Always convert the results to strings as late as possible, and try to use native token attributes to keep things consistent.

Part 2

Rewrite the code to use the native token attributes instead of lists of `token_texts` and `pos_tags`.

Loop over each `token` in the `doc` and check the `token.pos_` attribute.

Use `doc[token.i + 1]` to check for the next token and its `.pos_` attribute.

If a proper noun before a verb is found, print its `token.text`.

In [19]:
import spacy

nlp = spacy.load("en_core_web_sm")
# doc = nlp("Berlin looks like a nice city")
doc = nlp("Berlin looks like a nice city Berlin")

doc_len = len(doc)

for token in doc:
    # Check if the current token is a proper noun
    if token.pos_ == "PROPN":
        # Check if the next token is a verb
        if token.i + 1 < doc_len: # If the doc ends with a proper noun, doc[token.i + 1] will fail
            if doc[token.i + 1].pos_ == "VERB":
                print("Found proper noun before a verb:", token.text)

Found proper noun before a verb: Berlin


## Word vectors and semantic similarity

### Comparing semantic similarity

spaCy can compare two objects and predict how similar they are – for example, documents, spans or single tokens.

The `Doc`, `Token` and `Span` objects have a `.similarity` method that takes another object and returns a floating point number between 0 and 1, indicating how similar they are.

One thing that's very important: In order to use similarity, you need a larger spaCy pipeline that has word vectors included.

For example, the medium or large English pipeline – but not the small one. So if you want to use vectors, always go with a pipeline that ends in "md" or "lg". You can find more details on this in the [documentation](https://spacy.io/models).

- spaCy can compare two objects and predict similarity
- Doc.similarity(), Span.similarity() and Token.similarity()
- Take another object and return a similarity score (0 to 1)
- Important: needs a pipeline that has word vectors included, for example:
  - ✅ en_core_web_md (medium)
  - ✅ en_core_web_lg (large)
  - 🚫 NOT en_core_web_sm (small)

### Similarity examples (1)

Here's an example. Let's say we want to find out whether two documents are similar.

First, we load the medium English pipeline, "en_core_web_md".

We can then create two doc objects and use the first doc's similarity method to compare it to the second.

Here, a fairly high similarity score of 0.86 is predicted for "I like fast food" and "I like pizza".

The same works for tokens.

According to the word vectors, the tokens "pizza" and "pasta" are kind of similar, and receive a score of 0.7.

In [22]:
# Load a larger pipeline with vectors
nlp = spacy.load("en_core_web_md")

# Compare two documents
doc1 = nlp("I like fast food")
doc2 = nlp("I like pizza")
print(doc1.similarity(doc2))

0.8382381200790405


In [23]:
# Compare two tokens
doc = nlp("I like pizza and pasta")
token1 = doc[2]
token2 = doc[4]
print(token1.similarity(token2))

1.0


### Similarity examples (2)

You can also use the `similarity` methods to compare different types of objects.

For example, a document and a token.

Here, the similarity score is pretty low and the two objects are considered fairly dissimilar.

Here's another example comparing a span – "pizza and pasta" – to a document about McDonalds.

The score returned here is 0.61, so it's determined to be kind of similar.

In [24]:
# Compare a document with a token
doc = nlp("I like pizza")
token = nlp("soap")[0]

print(doc.similarity(token))

0.2274085134267807


In [25]:
# Compare a span with a document
span = nlp("I like pizza and pasta")[2:5]
doc = nlp("McDonalds sells burgers")

print(span.similarity(doc))

0.5528545379638672


### How does spaCy predict similarity?

But how does spaCy do this under the hood?

Similarity is determined using word vectors, multi-dimensional representations of meanings of words.

You might have heard of Word2Vec, which is an algorithm that's often used to train word vectors from raw text.

Vectors can be added to spaCy's pipelines.

By default, the similarity returned by spaCy is the cosine similarity between two vectors – but this can be adjusted if necessary.

Vectors for objects consisting of several tokens, like the `Doc` and `Span`, default to the average of their token vectors.

That's also why you usually get more value out of shorter phrases with fewer irrelevant words.

- Similarity is determined using word vectors
- Multi-dimensional meaning representations of words
- Generated using an algorithm like [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) and lots of text
- Can be added to spaCy's pipelines
- Default: cosine similarity, but can be adjusted
- `Doc` and `Span` vectors default to average of token vectors
- Short phrases are better than long documents with many irrelevant words

### Word vectors in spaCy

To give you an idea of what those vectors look like, here's an example.

First, we load the medium pipeline again, which ships with word vectors.

Next, we can process a text and look up a token's vector using the `.vector` attribute.

The result is a 300-dimensional vector of the word "banana".

In [26]:
# Load a larger pipeline with vectors
nlp = spacy.load("en_core_web_md")

doc = nlp("I have a banana")
# Access the vector via the token.vector attribute
print(doc[3].vector)

[-0.6334     0.18981   -0.53544   -0.52658   -0.30001    0.30559
 -0.49303    0.14636    0.012273   0.96802    0.0040354  0.25234
 -0.29864   -0.014646  -0.24905   -0.67125   -0.053366   0.59426
 -0.068034   0.10315    0.66759    0.024617  -0.37548    0.52557
  0.054449  -0.36748   -0.28013    0.090898  -0.025687  -0.5947
 -0.24269    0.28603    0.686      0.29737    0.30422    0.69032
  0.042784   0.023701  -0.57165    0.70581   -0.20813   -0.03204
 -0.12494   -0.42933    0.31271    0.30352    0.09421   -0.15493
  0.071356   0.15022   -0.41792    0.066394  -0.034546  -0.45772
  0.57177   -0.82755   -0.27885    0.71801   -0.12425    0.18551
  0.41342   -0.53997    0.55864   -0.015805  -0.1074    -0.29981
 -0.17271    0.27066    0.043996   0.60107   -0.353      0.6831
  0.20703    0.12068    0.24852   -0.15605    0.25812    0.007004
 -0.10741   -0.097053   0.085628   0.096307   0.20857   -0.23338
 -0.077905  -0.030906   1.0494     0.55368   -0.10703    0.052234
  0.43407   -0.13926    0

### Similarity depends on the application context

Predicting similarity can be useful for many types of applications. For example, to recommend a user similar texts based on the ones they have read. It can also be helpful to flag duplicate content, like posts on an online platform.

However, it's important to keep in mind that there's no objective definition of what's similar and what isn't. It always depends on the context and what your application needs to do.

Here's an example: spaCy's default word vectors assign a very high similarity score to "I like cats" and "I hate cats". This makes sense, because both texts express sentiment about cats. But in a different application context, you might want to consider the phrases as very dissimilar, because they talk about opposite sentiments.

- Useful for many applications: recommendation systems, flagging duplicates etc.
- There's no objective definition of "similarity"
- Depends on the context and what application needs to do

In [27]:
doc1 = nlp("I like cats")
doc2 = nlp("I hate cats")

print(doc1.similarity(doc2))

1.0


## Inspecting word vectors

In this exercise, you’ll use a larger [English pipeline](https://spacy.io/models/en), which includes around 20.000 word vectors. The pipeline package is already pre-installed.

- Load the medium `"en_core_web_md"` pipeline with word vectors.
- Print the vector for `"bananas"` using the `token.vector` attribute.

In [33]:
import spacy

# Load the en_core_web_md pipeline
nlp = spacy.load("en_core_web_md")

# Process a text
doc = nlp("Two bananas in pyjamas")

# Get the vector for the token "bananas"
bananas_vector = doc[1].vector
print(bananas_vector)

[-0.6334     0.18981   -0.53544   -0.52658   -0.30001    0.30559
 -0.49303    0.14636    0.012273   0.96802    0.0040354  0.25234
 -0.29864   -0.014646  -0.24905   -0.67125   -0.053366   0.59426
 -0.068034   0.10315    0.66759    0.024617  -0.37548    0.52557
  0.054449  -0.36748   -0.28013    0.090898  -0.025687  -0.5947
 -0.24269    0.28603    0.686      0.29737    0.30422    0.69032
  0.042784   0.023701  -0.57165    0.70581   -0.20813   -0.03204
 -0.12494   -0.42933    0.31271    0.30352    0.09421   -0.15493
  0.071356   0.15022   -0.41792    0.066394  -0.034546  -0.45772
  0.57177   -0.82755   -0.27885    0.71801   -0.12425    0.18551
  0.41342   -0.53997    0.55864   -0.015805  -0.1074    -0.29981
 -0.17271    0.27066    0.043996   0.60107   -0.353      0.6831
  0.20703    0.12068    0.24852   -0.15605    0.25812    0.007004
 -0.10741   -0.097053   0.085628   0.096307   0.20857   -0.23338
 -0.077905  -0.030906   1.0494     0.55368   -0.10703    0.052234
  0.43407   -0.13926    0

## Comparing similarities

In this exercise, you’ll be using spaCy’s `similarity` methods to compare `Doc`, `Token` and `Span` objects and get similarity scores.

Part 1

- Use the `doc.similarity` method to compare `doc1` to `doc2` and print the result.

In [34]:
import spacy

nlp = spacy.load("en_core_web_md")

doc1 = nlp("It's a warm summer day")
doc2 = nlp("It's sunny outside")

# Get the similarity of doc1 and doc2
similarity = doc1.similarity(doc2)
print(similarity)

0.8456854224205017


Part 2

- Use the `token.similarity` method to compare `token1` to `token2` and print the result.


In [35]:
import spacy

nlp = spacy.load("en_core_web_md")

doc = nlp("TV and books")
token1, token2 = doc[0], doc[2]

# Get the similarity of the tokens "TV" and "books"
similarity = token1.similarity(token2)
print(similarity)

0.18317238986492157


Part 3

- Create spans for “great restaurant”/“really nice bar”.

- Use `span.similarity` to compare them and print the result.

In [37]:
import spacy

nlp = spacy.load("en_core_web_md")

doc = nlp("This was a great restaurant. Afterwards, we went to a really nice bar.")

# Create spans for "great restaurant" and "really nice bar"
span1 = doc[3:5]
span2 = doc[-4:-1]

# Get the similarity of the spans
similarity = span1.similarity(span2)
print(similarity)

0.7541285157203674
